4. การทำนายการเสียของเครื่องจักร (Predictive Maintenance)
โจทย์ Classification ที่ต้องอาศัยทักษะการทำ Feature Engineering จากข้อมูลอนุกรมเวลา (Time Series)

🎯 โจทย์: โรงงานอุตสาหกรรมต้องการลด Downtime โดยมีข้อมูลจาก Sensor ของเครื่องจักร (อุณหภูมิ, แรงดัน, การสั่นสะเทือน) และข้อมูลการทำงาน โจทย์คือให้ทำนายว่าเครื่องจักรจะ "เสียในอนาคตอันใกล้" (Target=1) หรือไม่

🔢 ชุดข้อมูลที่คล้ายกันสำหรับฝึกซ้อม: **[Predictive Maintenance Dataset (AI4I 2020)](

🔢 ชุดข้อมูลที่คล้ายกันสำหรับฝึกซ้อม: Predictive Maintenance Dataset (AI4I 2020)

ทำไมถึงเหมาะ: เป็นชุดข้อมูลที่จำลองสถานการณ์จริงได้ดี มีทั้งข้อมูล Sensor และข้อมูลสถานะการทำงาน ซึ่งต้องนำมาสร้าง Feature ร่วมกัน

In [1]:
# ติดตั้ง AutoGluon และ library สำหรับดาวน์โหลดข้อมูลจาก Kaggle
# ขั้นตอนนี้อาจใช้เวลา 2-3 นาที
%pip install --upgrade "autogluon[all]" opendatasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [2]:
import opendatasets as od
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

# ระบุ URL ของชุดข้อมูลบน Kaggle
dataset_url = 'https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification'

# ดาวน์โหลดข้อมูล (ระบบจะถามหา Kaggle Username และ Key)
od.download(dataset_url)
# กำหนด Path ของไฟล์ข้อมูล
data_path = './machine-predictive-maintenance-classification/predictive_maintenance.csv'

# โหลดข้อมูลด้วย pandas
df = pd.read_csv(data_path)

# แสดงขนาดและตัวอย่างข้อมูลเพื่อตรวจสอบความถูกต้อง
print(f"ขนาดของข้อมูล: {df.shape}")
print("\nตัวอย่างข้อมูล 5 แถวแรก:")
df.head()


Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: xzcCs
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification


100%|██████████| 137k/137k [00:00<00:00, 121MB/s]


ขนาดของข้อมูล: (10000, 10)

ตัวอย่างข้อมูล 5 แถวแรก:


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [4]:
# แบ่งข้อมูลเป็น train (80%) และ test (20%) เพื่อจำลองการแข่งขัน
# random_state=42 เพื่อให้สามารถทำซ้ำผลลัพธ์เดิมได้ทุกครั้ง
train_data = df.sample(frac=0.8, random_state=42)
test_data = df.drop(train_data.index)

# เก็บ UDI ของ test_data ไว้สำหรับสร้างไฟล์ submission
test_ids = test_data['UDI']

# ลบคอลัมน์คำตอบ ('Machine failure') และคอลัมน์ที่ไม่จำเป็น (Product ID) ออกจาก test_data
test_data_no_label = test_data.drop(columns=['Target', 'Product ID'])

print(f"ขนาดข้อมูลสำหรับ Train: {train_data.shape}")
print(f"ขนาดข้อมูลสำหรับ Test:  {test_data_no_label.shape}")


ขนาดข้อมูลสำหรับ Train: (8000, 10)
ขนาดข้อมูลสำหรับ Test:  (2000, 8)


In [5]:
# กำหนดคอลัมน์เป้าหมาย
label = 'Target'

# กำหนดเวลาในการฝึก (หน่วยเป็นวินาที), 300 วินาที = 5 นาที
time_limit = 300

# สร้าง Predictor
# เราจะลบคอลัมน์ที่ไม่จำเป็นออกไประหว่างการฝึก
predictor = TabularPredictor(
    label=label,
    eval_metric='roc_auc',
    path='ag_models_maintenance'
)

# เริ่มฝึกโมเดล
# เราสามารถส่งข้อมูล train ทั้ง DataFrame เข้าไปได้เลย
# AutoGluon จะจัดการกับคอลัมน์ที่ไม่ใช่ feature หรือ label เอง (เช่น UDI, Product ID)
predictor.fit(
    train_data,
    time_limit=time_limit,
    presets='best_quality'
)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
Memory Avail:       11.29 GB / 12.67 GB (89.1%)
Disk Space Avail:   65.02 GB / 107.72 GB (60.4%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stack

(_ray_fit pid=2136) [1000]	valid_set's binary_logloss: 0.000112703
(_ray_fit pid=2136) [2000]	valid_set's binary_logloss: 7.10236e-05
(_ray_fit pid=2136) [3000]	valid_set's binary_logloss: 5.33677e-05
(_ray_fit pid=2136) [4000]	valid_set's binary_logloss: 4.66181e-05


(_ray_fit pid=2136) 	Ran out of time, early stopping on iteration 4497. Best iteration is:
(_ray_fit pid=2136) 	[4468]	valid_set's binary_logloss: 4.41649e-05
(_dystack pid=1956) 	0.9953	 = Validation score   (roc_auc)
(_dystack pid=1956) 	43.05s	 = Training   runtime
(_dystack pid=1956) 	0.89s	 = Validation runtime
(_dystack pid=1956) Fitting model: WeightedEnsemble_L2 ... Training model for up to 68.51s of the 15.65s of remaining time.
(_dystack pid=1956) 	Ensemble Weights: {'LightGBMXT_BAG_L1': 1.0}
(_dystack pid=1956) 	0.9953	 = Validation score   (roc_auc)
(_dystack pid=1956) 	0.13s	 = Training   runtime
(_dystack pid=1956) 	0.0s	 = Validation runtime
(_dystack pid=1956) Fitting 108 L2 models, fit_strategy="sequential" ...
(_dystack pid=1956) Fitting model: LightGBMXT_BAG_L2 ... Training model for up to 15.51s of the 15.49s of remaining time.
(_dystack pid=1956) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (2 workers, per: cpus=1, gpus=0, m

(_ray_fit pid=2803) [1000]	valid_set's binary_logloss: 0.000343169


(_ray_fit pid=2803) 	Ran out of time, early stopping on iteration 1695. Best iteration is:
(_ray_fit pid=2803) 	[1636]	valid_set's binary_logloss: 0.000222482
(_dystack pid=1956) 	0.996	 = Validation score   (roc_auc)
(_dystack pid=1956) 	35.47s	 = Training   runtime
(_dystack pid=1956) 	0.9s	 = Validation runtime
(_dystack pid=1956) Fitting model: WeightedEnsemble_L3 ... Training model for up to 68.51s of the -25.96s of remaining time.
(_dystack pid=1956) 	Ensemble Weights: {'LightGBMXT_BAG_L2': 0.818, 'LightGBMXT_BAG_L1': 0.182}
(_dystack pid=1956) 	0.9964	 = Validation score   (roc_auc)
(_dystack pid=1956) 	0.29s	 = Training   runtime
(_dystack pid=1956) 	0.02s	 = Validation runtime
(_dystack pid=1956) AutoGluon training complete, total runtime = 94.96s ... Best model: WeightedEnsemble_L3 | Estimated inference throughput: 489.8 rows/s (889 batch size)
(_dystack pid=1956) TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/content/ag_models_maintenance/ds_sub_fi

In [6]:
# แสดง Leaderboard ของโมเดลทั้งหมด
# สำหรับ ROC AUC ค่ายิ่งสูงยิ่งดี
predictor.leaderboard(train_data, silent=True)


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,1.000000,0.748787,roc_auc,0.103458,0.045790,0.011743,0.103458,0.045790,0.011743,1,True,2
1,RandomForestEntr_BAG_L1,1.000000,0.987201,roc_auc,0.140131,0.277020,2.005469,0.140131,0.277020,2.005469,1,True,6
2,RandomForestGini_BAG_L1,1.000000,0.987294,roc_auc,0.151896,0.409789,2.806537,0.151896,0.409789,2.806537,1,True,5
3,CatBoost_BAG_L1,1.000000,0.994085,roc_auc,0.157397,0.084582,74.944688,0.157397,0.084582,74.944688,1,True,7
4,ExtraTreesEntr_BAG_L1,1.000000,0.986965,roc_auc,0.183300,0.464424,2.102337,0.183300,0.464424,2.102337,1,True,9
5,ExtraTreesGini_BAG_L1,1.000000,0.990431,roc_auc,0.199347,0.490577,1.744494,0.199347,0.490577,1.744494,1,True,8
6,LightGBM_BAG_L1,1.000000,0.992386,roc_auc,0.761084,0.165586,37.228423,0.761084,0.165586,37.228423,1,True,4
7,LightGBMXT_BAG_L1,1.000000,0.994997,roc_auc,7.728765,0.705862,37.037006,7.728765,0.705862,37.037006,1,True,3
8,WeightedEnsemble_L2,1.000000,0.994997,roc_auc,7.730998,0.708204,37.621197,0.002233,0.002342,0.584191,2,True,11
9,NeuralNetFastAI_BAG_L1,0.986245,0.988118,roc_auc,1.313834,0.742060,51.886756,1.313834,0.742060,51.886756,1,True,10


In [7]:
# ทำนายผลบนข้อมูล test
predictions = predictor.predict(test_data_no_label)

# สร้าง DataFrame สำหรับไฟล์ submission
submission = pd.DataFrame({
    'UDI': test_ids,
    'Machine failure': predictions
})

# แสดงตัวอย่าง 5 แถวแรกของไฟล์ submission
print("ตัวอย่างไฟล์ Submission:")
submission.head()

# บันทึกเป็นไฟล์ CSV
submission.to_csv('submission_maintenance.csv', index=False)

print("\nสร้างไฟล์ submission_maintenance.csv เรียบร้อยแล้ว!")


ตัวอย่างไฟล์ Submission:

สร้างไฟล์ submission_maintenance.csv เรียบร้อยแล้ว!


In [9]:
submission['Machine failure'].value_counts()

,count
Machine failure,
0,1927
1,73


In [10]:
submission

,UDI,Machine failure
1,2,0
4,5,0
5,6,0
9,10,0
11,12,0
...,...,...
9984,9985,0
9986,9987,0
9987,9988,0
9988,9989,0
